In [1]:
import os
import gym
import numpy as np

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.callbacks import BaseCallback

from Enviroment import Enviroment
from gym import spaces
import cv2
from tqdm import tqdm

from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
import torch.nn.functional as F
import torch
from stable_baselines3 import PPO

pygame 2.0.1 (SDL 2.0.14, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
torch.backends.cudnn.benchmark = True

# Callback class

In [3]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """
    def __init__(self, check_freq: int, log_dir: str, verbose=1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model_PPO')
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        pass
        # Create folder if needed
#         if self.save_path is not None:
#             os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

          # Retrieve training reward
          x, y = ts2xy(load_results(self.log_dir), 'timesteps')
          if len(x) > 0:
              # Mean training reward over the last 100 episodes
              mean_reward = np.mean(y[-100:])
              if self.verbose > 0:
                print("Num timesteps: {}".format(self.num_timesteps))
                print("Best mean reward: {:.2f} - Last mean reward per episode: {:.2f}".format(self.best_mean_reward, mean_reward))

              # New best model, you could save the agent here
              if mean_reward > self.best_mean_reward:
                  self.best_mean_reward = mean_reward
                  # Example for saving best model
                  if self.verbose > 0:
                    print("Saving new best model to {}".format(self.save_path))
                  self.model.save(self.save_path)

        return True


# Neural network class

In [4]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class CustomCNN(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """

    def __init__(self, observation_space: gym.spaces.Dict, features_dim: int = 518):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        
        extractors = {}
        
        for key, subspace in observation_space.spaces.items():
            if key == "img":
        
                n_input_channels = observation_space[key].shape[0]
            
                extractors[key] = nn.Sequential(

                nn.Conv2d(n_input_channels, 32, 2),
                nn.MaxPool2d(2, 2),
                nn.Conv2d(32, 64, 2),
                nn.MaxPool2d(2, 2),

                ResBlock(n_filters=64, kernel_size=2),
                nn.MaxPool2d(4, 4),
                ResBlock(n_filters=64, kernel_size=2),
                nn.MaxPool2d(2, 2),
                ResBlock(n_filters=64, kernel_size=2),
                nn.MaxPool2d(2, 2),
                ResBlock(n_filters=64, kernel_size=2), 
                nn.MaxPool2d(2, 2),
                
                nn.Conv2d(64, 128, 2),
                nn.Flatten())
                    
            elif key == "posRobot":
                
                n_input_channels = observation_space[key].shape[0]
                
                extractors[key] = nn.Sequential(nn.Linear(n_input_channels, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 3))
#             elif key == "posRobot_2":
                
#                 n_input_channels = observation_space[key].shape[0]
                
#                 extractors[key] = nn.Sequential(nn.Linear(n_input_channels, 9),
#                                         nn.ReLU(),
#                                         nn.Linear(9, 9),
#                                         nn.ReLU(),
#                                         nn.Linear(9, 3))
                    
            elif key == "target":
                            
                n_input_channels = observation_space[key].shape[0]
                    
                extractors[key] = nn.Sequential(nn.Linear(n_input_channels, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 3))
#             elif key == "target_2":
                            
#                 n_input_channels = observation_space[key].shape[0]
                    
#                 extractors[key] = nn.Sequential(nn.Linear(n_input_channels, 9),
#                                         nn.ReLU(),
#                                         nn.Linear(9, 9),
#                                         nn.ReLU(),
#                                         nn.Linear(9, 3))
                
        self.extractors = nn.ModuleDict(extractors)

    def forward(self, observations: th.Tensor) -> th.Tensor:
        '''
        Forward propagation
        :param observations: (dict) изображение; координаты и углы ориентации агентов
        :return: features tensor
        '''
        encoded_tensor_list = []

        for key, extractor in self.extractors.items():
            encoded_tensor_list.append(extractor(observations[key]))

        return th.cat(encoded_tensor_list, dim=1)

    
class ResBlock(nn.Module):
    def __init__(self, n_filters, kernel_size):
        """
        Инициализация кастомного резнетовского блока
        :param n_filters: (int) количество фильтров сверточного слоя
        :param kernel_size: (int) размер ядра свертки
        """
        super().__init__()
        self.n_filters = n_filters
        self.kernel_size = kernel_size

        self.b1 = nn.Conv2d(self.n_filters, self.n_filters, self.kernel_size, padding='same')
    
        self.b2 = nn.BatchNorm2d(self.n_filters, eps = 0.001, momentum= 0.99)
        self.b3 = nn.Conv2d(self.n_filters, self.n_filters, self.kernel_size, padding='same')
        self.b4 = nn.BatchNorm2d(self.n_filters, eps = 0.001, momentum= 0.99)
        
    def forward(self, x):
        '''
        Forward propagation
        :param x: input
        :return: output
        '''
        residual = x
        y = F.relu(self.b1(x))
        y = self.b2(y)
        y = F.relu(self.b3(y))
        y = self.b4(y)
        y += residual
        y = F.relu(y)
        return y


# Environment gym class

In [5]:
class CustomEnv(gym.Env):
    '''
    Оборочивание класса среды в среду gym
    '''
    metadata = {'render.modes': ['human']}

    def __init__(self, obstacle_turn: bool, Total_war: bool,
                 num_obs: int, num_enemy: int, num_alias: int, 
                 size_obs, steps_limit, vizualaze=False, head_velocity=0.01):
        '''
        Инициализация класса среды
        :param obstacle_turn: (bool) Флаг генерации препятствий
        :param vizualaze: (bool) Флаг генерации препятствий
        :param Total_war: (bool) Флаг режима игры (с противником или без)
        :param steps_limit: (int) Максимальное количество действий в среде за одну игру
        '''
        self.log_koef = 50

        self.velocity_coef = 35       #  1/2 max speed !!!
        self.ang_Norm_coef = np.pi
        self.coords_Norm_coef = 500
        self.n_alias = num_alias
        self.num_enemies = num_enemy
        
        self.enviroment = Enviroment(obstacle_turn, vizualaze, Total_war, head_velocity, num_obs, num_enemy,
                         num_alias, size_obs, steps_limit)

        state = self.enviroment.reset()

        self.action_space = spaces.Box(low=np.array([-1, -1,-1, -1]), high=np.array([1, 1, 1, 1]), dtype=np.float16)
        self.observation_space = gym.spaces.Dict({
                    'img': spaces.Box(low=0, high=255, shape=(500, 500, 3), dtype=np.uint8),
            
                    'posRobot': spaces.Box(low  = np.array([0, 0,-0.1,0, 0,-0.1]), 
                                           high = np.array([1, 1, 1,1, 1, 1]),
                                           dtype = np.float32),
                                                
                    'target': spaces.Box(low  = np.array([0, 0,-0.1,0, 0,-0.1]), 
                                           high = np.array([1, 1, 1,1, 1, 1]),
                                           dtype = np.float32)
                                                })
        
        self.a_f = False
        self.e_f = False
        
        self.changecoordsA = lambda x: x
        self.changecoordsE = lambda x: x
        
        self.img1 = state.img
        self.img2 = state.img
        self.img3 = state.img
        self.Img = None
        

    def make_layers(self):
        """
        Функция наслоения изображений трех последовательных шагов в среде
        :param img1, img2, img3: состояния среды на трех последовательных шагах
        :return: new_img: изображение, содержащее информацию о состояниях среды на трех последовательных шагах, отображенную с разной интенсивностью
        """
        new_img = cv2.addWeighted(self.img2, 0.4, self.img1, 0.2, 0)
        self.Img = cv2.addWeighted(self.img3, 0.7, new_img, 0.5, 0)
    
    
    def step(self, action):
        """
        Метод осуществления шага в среде
        :param action: (int) направление движения в среде
        :return: dict_state, reward, not done, {}: состояние, реворд, флаг терминального состояния, информация о среде
        """
        
        action[0] *= self.velocity_coef
        action[1] *= self.ang_Norm_coef
        
        action[2] *= self.velocity_coef
        action[3] *= self.ang_Norm_coef
        
        action = np.array([action[:2],
                          action[2:]])
        
        state, reward, done, numstep = self.enviroment.step(action)
        
        self.img1 = self.img2
        self.img2 = self.img3
        self.img3 = state.img
        
        self.make_layers()

        f1, f2 = state.posRobot[0,2], state.posRobot[1,2] # угол первого и второго ртп
        
        x1, x2 = state.posRobot[0,0], state.posRobot[1,0]
        y1, y2 = state.posRobot[0,1], state.posRobot[1,1]

        l1, l2 = not bool(state.posRobot[0,3]), not bool(state.posRobot[1,3])
        l3, l4 = not bool(state.target[0,3]),   not bool(state.target[1,3])

        x3, x4 = state.target[0,0], state.target[1,0]
        y3, y4 = state.target[0,1], state.target[1,1]

        dist_matrix = np.array([[np.sqrt(np.abs((x1-x3)**2 - (y1-y3)**2))],
                                [np.sqrt(np.abs((x1-x4)**2 - (y1-y4)**2))],
                                [np.sqrt(np.abs((x2-x3)**2 - (y2-y3)**2))],
                                [np.sqrt(np.abs((x2-x4)**2 - (y2-y4)**2))]])

        live_matrix = np.array([[l1*l3],
                                [l1*l4],
                                [l2*l3],
                                [l2*l4]])

        Ax3, Ay3, Ax4, Ay4 = -np.cos(f1), np.sin(f1), -np.cos(f2), np.sin(f2)

        Bx13, By13, Bx14, By14 = x1 - x3, y1 - y3, x1 - x4, y1 - y4        
        Bx23, By23, Bx24, By24 = x2 - x3, y2 - y3, x2 - x4, y2 - y4

        phy = np.arccos([[(Ax3*Bx13 + Ay3*By13)/(np.sqrt(Ax3**2 + Ay3**2) * np.sqrt(Bx13**2 + By13**2))],
                         [(Ax4*Bx14 + Ay4*By14)/(np.sqrt(Ax4**2 + Ay4**2) * np.sqrt(Bx14**2 + By14**2))],
                         [(Ax3*Bx23 + Ay3*By23)/(np.sqrt(Ax3**2 + Ay3**2) * np.sqrt(Bx23**2 + By23**2))],
                         [(Ax4*Bx24 + Ay4*By24)/(np.sqrt(Ax4**2 + Ay4**2) * np.sqrt(Bx24**2 + By24**2))]])
        
        distanse_reward = live_matrix * dist_matrix
        phy_matrix = phy * live_matrix 
        
        reward_l = np.log2(np.sum(phy_matrix) * 40  / np.sum(distanse_reward)) #* 900
        reward_l = np.nan_to_num(reward_l * np.sum(live_matrix)) + np.sum(reward)
        
        dict_state = {'img':     self.Img,  
                      'posRobot':self.norm(state.target[:,:3].reshape(-1)),
                      'target':  self.norm(state.target[:,:3].reshape(-1)),
                     }
       

        return dict_state, reward_l, done, {}
    
    def norm(self, coords):
        '''
        Метод нормализации координат
        :return: coords: нормализованные координаты
        '''
        coords=np.float32(coords)
        coords[2]  = coords[2] / self.ang_Norm_coef #угол
        coords[:2] = coords[:2] / self.coords_Norm_coef #координаты
        
        return coords


    def reset(self):
        '''
        Метод обновления игры
        :return: dict_state: состояние
        '''
        
        state = self.enviroment.reset()
        
        self.img2 = state.img
        self.img3 = state.img
        
        self.Img = state.img
        
        # state.posRobot = state.posRobot[:,:3]
        # state.target = state.target[:,:3]
        
        
        dict_state = {'img':     self.Img,  
                      'posRobot':self.norm(state.target[:,:3].reshape(-1)),
                      'target':  self.norm(state.target[:,:3].reshape(-1)),
                     }


        return dict_state
    

    def render(self, model, num_gifs=1):
        '''
        Метод вывода информации об игре
        :param mode:
        :return:
        '''
        for i in range(num_gifs):
            
            images = []
            obs = self.reset()
            img = obs['img']# env.render(mode='rgb_array')
            done = False
                
            height, width, layers = img.shape
            size = (width,height)
            out = cv2.VideoWriter(f"video{i}.avi",cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            out.write(img)
            while not done:

                action, _ = model.predict(obs)
                obs, _, done ,_ = self.step(action)
                img = obs['img']
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                out.write(img)
            out.release()
    
    def get_statistic(self, model, num_games):
        collision = 0
        win = 0
        destroyed = 0
        loss = 0
        
        pbar = tqdm(range(num_games))
        for i in pbar:
            obs = self.reset()
            done = False
            while not done:
                action, _ = model.predict(obs)
                obs, reward, done ,_ = self.step(action)   
                
            if reward == 100:# loss
                win +=1
            elif reward == -100:# loss
                destroyed +=1
            else:    #not_achieved
                loss+=1
        
        print("Win: ",win/num_games)
        print("destroyed: ", destroyed/num_games)
        print("loss: ",loss/num_games)
        print("collision: ",collision/num_games)


# Initialize the enviroment

In [6]:
# env = CustomEnv(obstacle_turn = False,
#                 vizualaze     = True, 
#                 Total_war     = True,
#                 head_velocity = 0.05,
#                 num_obs       = 5, 
#                 num_enemy     = 2,
#                 num_alias     = 2,
#                 size_obs      = [30, 40],
#                 steps_limit   = 300)

In [7]:
# f = np.pi/10
# action = [1,f,1,f]

# obs, reward, done, inf = env.step(action)
# print(reward, done)

In [8]:
env = CustomEnv(obstacle_turn = False,
            vizualaze     = False, 
            Total_war     = True,
            head_velocity = 0.005,
            num_obs       = 5, 
            num_enemy     = 2,
            num_alias     = 2,
            size_obs      = [30, 40],
            steps_limit   = 5000)

/home/user/anaconda3/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float16
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/home/user/anaconda3/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


# Initialize the agent

In [9]:
from stable_baselines3 import PPO, A2C, TD3, DDPG, SAC

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=518),
    activation_fn=torch.nn.ReLU,
    net_arch = [524, 128, 32, 8])
                     # , vf=[1029, 128, 32, 8])])

model = PPO(policy          = 'MultiInputPolicy',
            env             = env,
            # learning_rate   = 0.0001,
            n_steps         = 24,
            batch_size      = 24,
            gamma           = 0.99,
            gae_lambda      = 0.95,
            tensorboard_log = "./tensorboard_logs/",
            policy_kwargs   = policy_kwargs,
            verbose         = 1,
            device          = 'cuda')

# model = A2C(policy          = 'MlpPolicy',
#             env             = env,
#             learning_rate   = 0.0001,
#             n_steps         = 24,
#             gamma           = 0.99,
#             gae_lambda      = 0.95,
#             tensorboard_log = "./tensorboard_logs/",
#             policy_kwargs   = policy_kwargs,
#             verbose         = 1,
#             device          = 'cuda')
# model = TD3(policy          = 'MultiInputPolicy',  # 1 neural network metod
#             env             = env,
#             learning_rate   = 0.0001,
#             buffer_size     = 10000,
#             batch_size      = 40,
#             gamma           = 0.99,
#             tensorboard_log = "./tensorboard_logs_cont_mult/",
#             policy_kwargs   = policy_kwargs,
#             verbose         = 1,
#             device          = 'cuda')

# model = DDPG(policy         = 'MlpPolicy',  # 1 neural network metod
#             env             = env,
#             learning_rate   = 0.0001,
#             buffer_size     = 100,
#             batch_size      = 2,
#             gamma           = 0.99,
#             tensorboard_log = "./tensorboard_logs_cont_mult/",
#             policy_kwargs   = policy_kwargs,
#             verbose         = 0,
#             device          = 'cuda')

# model = SAC(policy          = 'MlpPolicy',  # 1 neural network metod
#             env             = env,
#             learning_rate   = 0.0001,
#             buffer_size     = 100,
#             batch_size      = 2,
#             gamma           = 0.99,
#             tensorboard_log = "./tensorboard_logs_cont_mult/",
#             policy_kwargs   = policy_kwargs,
#             verbose         = 0,
#             device          = 'cuda')

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


# Make callback

In [10]:
log_dir = './saved_models/PPO/callback/'  # For A2C agent: './saved_models/A2C/callback'
os.makedirs(log_dir, exist_ok=True)
env = Monitor(env, log_dir)
callback = SaveOnBestTrainingRewardCallback(check_freq=500, log_dir=log_dir)

# Learn model

In [ ]:
model.learn(total_timesteps=1e6,callback=callback)

Logging to ./tensorboard_logs/PPO_4


/home/user/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/user/anaconda3/lib/python3.8/site-packages/torch/nn/modules/conv.py:439: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/aten/src/ATen/native/Convolution.cpp:660.)
  return F.conv2d(input, weight, bias, self.stride,


---------------------------
| time/              |    |
|    fps             | 16 |
|    iterations      | 1  |
|    time_elapsed    | 1  |
|    total_timesteps | 24 |
---------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 16         |
|    iterations           | 2          |
|    time_elapsed         | 2          |
|    total_timesteps      | 48         |
| train/                  |            |
|    approx_kl            | 0.03574024 |
|    clip_fraction        | 0.388      |
|    clip_range           | 0.2        |
|    entropy_loss         | -5.68      |
|    explained_variance   | -2.03      |
|    learning_rate        | 0.0003     |
|    loss                 | 1.08       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0404    |
|    std                  | 1          |
|    value_loss           | 75.1       |
----------------------------------------
-------------------------

# Make environment to test trained model and get statistics

In [ ]:
# env_test = CustomEnv(obstacle_turn = False,
#             vizualaze     = False, 
#             Total_war     = True,
#             head_velocity = 0.005,
#             num_obs       = 5, 
#             num_enemy     = 2,
#             num_alias     = 2,
#             size_obs      = [30, 40],
#             steps_limit   = 2000)

# Load the best model and get statistics

In [ ]:
# path = './saved_models/PPO/callback0/best_model_PPO/'  # For A2C agent: './saved_models/A2C/callback0/best_model_A2C/'
# model = PPO.load(path, env=env_test)  # For A2C agent: A2C.load(path, env=env_test)

In [ ]:
# env_test.get_statistic(model, 10000)

# Make video

In [ ]:
# env_test.render(model = model, num_gifs=10)

# Check tensorboard

In [ ]:
# !tensorboard --logdir ./tensorboard_logs_cont_mult/